In [6]:
# Imports:
import pandas as pd
import numpy as np
# non-library stuff
from utils import *

# Retrieve Data
data = retrieve_data()
train = data['train']
test = data['test']
train_num = data['train_num']
y_feature = 'SalePrice'

1. Preprocess:
    - Impute missing numerical values
    - Encode categorical variables 
    - Try to accordingly (to their discrption) preprocess data
2. Find the best features by running linear regression on the different combinations of data

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [4]:
True in impute_numerical(train).isna().any()

False

In [5]:
# LotFrontage and GarageYrBlt should equal zeos 
# for NaN's since it means 0 meters of lot frontage
train['LotFrontage'] = train['LotFrontage'].fillna(0)
train['MasVnrArea'] = train['MasVnrArea'].fillna(0)
train['GarageYrBlt'] = fillNaWithKNN('GarageYrBlt', data['train'].copy(), 'SalePrice')

# The missing numerical features
# data['train_num_missing']

# Cheking to see if the imputations have worked or not
True in train[data['train_num_list']].isna().any()

False

In [33]:
# testing KNNIMputator module
from KNNImputator import KKNImputator

In [ ]:
knn = KKNImputator()